In [14]:
import pandas as pd
import numpy as np
#import sframe
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingClassifier
import json
from __future__ import division


In [2]:
dtype_dict = {'id':int, 'member_Id':int, 'loan_amnt':float, 'funded_amnt':float, 'funded_amnt_inv':float, 
              'term':str, 'float_rate':float, 'installment':float, 'grade':str, 'sub_grade':str, 'emp_title':str,
              'emp_length':str, 'home_ownership':str, 'annual_inc':float, 'is_inc_v':str, 'issue_d':str, 'loan_status':str, 
              'pymnt_plan':str, 'url':str, 'desc':str, 'purpose':str,'title':str,'zipcode':str,'addr_state':str,'dti':float,'deling_2yrs':float,'earliest_cr_line':str,
             'inq_last_6mths':float,'mths_since_last_delinq':float,'mths_since_last_record':float,'open_acc':float,'pub_rec':float,
             'revol_bal':float,'revol_util':float,'total_acc':float,'initial_list_status':str,'out_prncp':float,'out_prncp_inv':float,
             'total_pymnt':float,'total_pymnt_inv':float,'total_rec_prncp':float,'total_rec_float':float,'total_rec_late_fee':float,
              'recoveries':float,'collection_recovery_fee':float,'last_pymnt_d':str,'last_pymnt_amnt':float,'next_pymnt_d':str,
              'last_credit_pull_d':str,'collections_12_mths_ex_med':float,'mths_since_last_major_derog':float,
              'policy_code':float,'not_compliant':float,'status':str,'inactive_loans':float,'bad_loans':int,'emp_length_num':float,
              'grade_num':float,'sub_grade_num':float,'delinq_2yrs_zero':float,'pub_rec_zero':float,'collections_12_mths_zero':float,
              'short_emp':float,'payment_inc_ratio':float,'final_d':str,'last_delinq_none':float,'last_record_none':float,'last_major_derog_none':float}
#str,float,int

In [3]:
loans = pd.read_csv("/Users/wxu/workspace/ML-UW-Coursera/classification/week5/lending-club-data.csv",header=0,dtype = dtype_dict)
loans["safe_loans"] = loans['bad_loans'].apply(lambda x: 1 if x==0 else -1)
loans.drop("bad_loans",1)
loans.dtypes

id                               int64
member_id                        int64
loan_amnt                      float64
funded_amnt                    float64
funded_amnt_inv                float64
term                            object
int_rate                       float64
installment                    float64
grade                           object
sub_grade                       object
emp_title                       object
emp_length                      object
home_ownership                  object
annual_inc                     float64
is_inc_v                        object
issue_d                         object
loan_status                     object
pymnt_plan                      object
url                             object
desc                            object
purpose                         object
title                           object
zip_code                        object
addr_state                      object
dti                            float64
delinq_2yrs              

In [4]:
target = 'safe_loans'
features = ['grade',                     # grade of the loan (categorical)
            'sub_grade_num',             # sub-grade of the loan as a number from 0 to 1
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'payment_inc_ratio',         # ratio of the monthly payment to income
            'delinq_2yrs',               # number of delinquincies
             'delinq_2yrs_zero',          # no delinquincies in last 2 years
            'inq_last_6mths',            # number of creditor inquiries in last 6 months
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'open_acc',                  # number of open credit accounts
            'pub_rec',                   # number of derogatory public records
            'pub_rec_zero',              # no derogatory public records
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
            'int_rate',                  # interest rate of the loan
            'total_rec_int',             # interest received to date
            'annual_inc',                # annual income of borrower
            'funded_amnt',               # amount committed to the loan
            'funded_amnt_inv',           # amount committed by investors for the loan
            'installment',               # monthly payment owed by the borrower
           ]


In [8]:
loans = loans[[target] + features].dropna()
loans.shape

(122578, 25)

In [9]:
#how to filter out categorical features and get dummy variable
categorical_variables = []
for feat_name, feat_type in zip(loans.columns, loans.dtypes):
    if feat_type == object:
        categorical_variables.append(feat_name)
print categorical_variables
loansWithDummy = pd.get_dummies(loans,columns = categorical_variables)
loans = loansWithDummy.fillna(0)

['grade', 'home_ownership', 'purpose']


In [70]:
with open("/Users/wxu/workspace/ML-UW-Coursera/classification/week5/train.json") as myjson:
    for line in myjson:
        trainIdx = json.loads(line)
with open("/Users/wxu/workspace/ML-UW-Coursera/classification/week5/valid.json") as myjson:
    for line in myjson:
        validIdx = json.loads(line)
train = loansWithDummy.iloc[trainIdx]
valid = loansWithDummy.iloc[validIdx]

In [16]:
validation_safe_loans = valid[valid[target] == 1]
validation_risky_loans = valid[valid[target] == -1]

sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)


,safe_loans,sub_grade_num,short_emp,emp_length_num,dti,payment_inc_ratio,delinq_2yrs,delinq_2yrs_zero,inq_last_6mths,last_delinq_none,...,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding
22,1,0.2,0,3,29.44,6.30496,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
26,1,0.6,1,1,12.19,13.49520,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
24,-1,0.4,0,3,13.97,2.96736,3,0,0,0,...,0,0,0,0,0,0,1,0,0,0
41,-1,1.0,0,11,16.33,1.90524,0,1,0,1,...,1,0,0,0,0,0,0,0,0,0


In [28]:
def get_numpy_data(df,features,label):
    # df: a data frame that need to be transformed
    # features: a list of strings for the features name
    # label : single string for the class labels
    features_frame = df[features]
    features_matrix = features_frame.as_matrix()
    label_sarray = df[[label]]
    label_array= label_sarray.as_matrix()
    label_array.shape = (len(label_array),)
    return(features_matrix,label_array)

In [29]:
modelFeature = list(loans.columns)
modelFeature.remove("safe_loans")
print modelFeature

['sub_grade_num', 'short_emp', 'emp_length_num', 'dti', 'payment_inc_ratio', 'delinq_2yrs', 'delinq_2yrs_zero', 'inq_last_6mths', 'last_delinq_none', 'last_major_derog_none', 'open_acc', 'pub_rec', 'pub_rec_zero', 'revol_util', 'total_rec_late_fee', 'int_rate', 'total_rec_int', 'annual_inc', 'funded_amnt', 'funded_amnt_inv', 'installment', 'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G', 'home_ownership_MORTGAGE', 'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT', 'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation', 'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase', 'purpose_medical', 'purpose_moving', 'purpose_other', 'purpose_small_business', 'purpose_vacation', 'purpose_wedding']


In [32]:
train_feature, train_label = get_numpy_data(train,modelFeature,target)
valid_feature, valid_label = get_numpy_data(valid,modelFeature,target)
sample_valid_feature,sample_valid_label = get_numpy_data(sample_validation_data,modelFeature,target)

In [24]:
clf = ensemble.GradientBoostingClassifier( max_depth=6,n_estimators=5)
model_5 = clf.fit(train_feature, train_label)




In [36]:
predSample = model_5.predict(sample_valid_feature)
acc = (predSample==sample_valid_label).sum()/len(predSample)
acc
predSample_prob = model_5.predict_proba(sample_valid_feature)
print predSample
print predSample_prob

[ 1  1 -1  1]
[[ 0.41642331  0.58357669]
 [ 0.46949689  0.53050311]
 [ 0.53807792  0.46192208]
 [ 0.39591639  0.60408361]]


In [39]:
predValid = model_5.predict(valid_feature)
print predValid
accValid = (predValid==valid_label).sum()/len(predValid)
print accValid

[-1  1 -1 ..., -1 -1 -1]
0.661460577337


In [45]:
fp = ((predValid==1) & (valid_label==-1)).sum()
fn = ((predValid==-1) & (valid_label==+1)).sum()
print fp
print fn

1652
1491


In [46]:
10000 * 1491  + 20000 * 1652

47950000

In [72]:
#valid["prediction"] = model_5.predict_proba(valid_feature)[:,1]
predValid=model_5.predict_proba(valid_feature)[:,1]
#predValid.shape=(9284,1)
#add a new column
valid.loc[:,"prediction"] = pd.Series(predValid,index = valid.index)

In [81]:
pd.set_option('display.max_columns', None)
validSortHighToLow = valid.sort_values(["prediction"],ascending=False)
validSortHighToLow.iloc[0:5,]




,safe_loans,sub_grade_num,short_emp,emp_length_num,dti,payment_inc_ratio,delinq_2yrs,delinq_2yrs_zero,inq_last_6mths,last_delinq_none,last_major_derog_none,open_acc,pub_rec,pub_rec_zero,revol_util,total_rec_late_fee,int_rate,total_rec_int,annual_inc,funded_amnt,funded_amnt_inv,installment,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,home_ownership_MORTGAGE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,prediction
8021,-1,0.4,0,4,12.73,12.16700,0,1,1,1,1,6,0,1,30.5,18.2281,5.99,1202.30,36000,12000,12000,365.01,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.673059
105247,1,0.4,0,4,7.81,11.82810,0,1,0,1,1,7,0,1,8.8,0.0000,6.62,3157.40,93450,30000,30000,921.11,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.661468
16736,1,0.4,0,8,8.87,4.44560,0,1,0,1,1,7,0,1,24.7,0.0000,5.79,912.37,83500,10200,10175,309.34,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0.661468
96975,1,0.2,0,11,15.74,8.95065,0,1,1,1,1,6,0,1,11.8,0.0000,6.03,566.24,71000,17400,17400,529.58,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0.661468
4690,1,0.2,0,6,9.07,5.21760,0,1,0,1,1,8,0,1,15.5,0.0000,6.03,902.19,70000,10000,10000,304.36,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0.661468


In [82]:
validSortLowToHigh = valid.sort_values(["prediction"])
validSortLowToHigh.iloc[0:5,]

,safe_loans,sub_grade_num,short_emp,emp_length_num,dti,payment_inc_ratio,delinq_2yrs,delinq_2yrs_zero,inq_last_6mths,last_delinq_none,last_major_derog_none,open_acc,pub_rec,pub_rec_zero,revol_util,total_rec_late_fee,int_rate,total_rec_int,annual_inc,funded_amnt,funded_amnt_inv,installment,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,home_ownership_MORTGAGE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,prediction
58794,-1,0.8,0,2,8.66,17.62510,0,1,0,1,1,8,0,1,64.4,92.520,16.29,2741.40,42000,17475,17475,616.88,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0.307334
84508,-1,0.8,1,1,7.37,16.62070,0,1,1,1,1,8,0,1,32.6,24.930,16.29,2777.21,36000,14125,14125,498.62,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0.307334
27502,-1,1.0,0,3,8.53,14.62800,0,1,0,1,1,6,0,1,79.5,54.709,14.22,492.33,45000,16000,16000,548.55,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.312806
114781,-1,0.4,0,3,21.90,7.18760,1,0,4,0,1,20,0,1,63.4,0.000,17.57,0.00,60000,10000,10000,359.38,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.315969
84921,-1,0.8,0,9,8.54,7.48113,0,1,2,0,1,8,0,1,51.3,0.000,16.29,0.00,36240,6400,6400,225.93,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.315969


In [77]:
validSortHighToLow.columns

Index([u'safe_loans', u'sub_grade_num', u'short_emp', u'emp_length_num',
       u'dti', u'payment_inc_ratio', u'delinq_2yrs', u'delinq_2yrs_zero',
       u'inq_last_6mths', u'last_delinq_none', u'last_major_derog_none',
       u'open_acc', u'pub_rec', u'pub_rec_zero', u'revol_util',
       u'total_rec_late_fee', u'int_rate', u'total_rec_int', u'annual_inc',
       u'funded_amnt', u'funded_amnt_inv', u'installment', u'grade_A',
       u'grade_B', u'grade_C', u'grade_D', u'grade_E', u'grade_F', u'grade_G',
       u'home_ownership_MORTGAGE', u'home_ownership_OTHER',
       u'home_ownership_OWN', u'home_ownership_RENT', u'purpose_car',
       u'purpose_credit_card', u'purpose_debt_consolidation',
       u'purpose_home_improvement', u'purpose_house',
       u'purpose_major_purchase', u'purpose_medical', u'purpose_moving',
       u'purpose_other', u'purpose_small_business', u'purpose_vacation',
       u'purpose_wedding', u'prediction'],
      dtype='object')

In [83]:
clf10 = ensemble.GradientBoostingClassifier( max_depth=6,n_estimators=10)
model_10 = clf10.fit(train_feature, train_label)
clf50 = ensemble.GradientBoostingClassifier( max_depth=6,n_estimators=50)
model_50 = clf50.fit(train_feature, train_label)
clf100 = ensemble.GradientBoostingClassifier( max_depth=6,n_estimators=100)
model_100 = clf100.fit(train_feature, train_label)
clf200 = ensemble.GradientBoostingClassifier( max_depth=6,n_estimators=200)
model_200 = clf200.fit(train_feature, train_label)
clf500 = ensemble.GradientBoostingClassifier( max_depth=6,n_estimators=500)
model_500 = clf500.fit(train_feature, train_label)


In [93]:
def acc(model,df_features,df_target):
    pred=model.predict(df_features)
    acc=(pred==df_target).sum()/len(pred)
    return acc

In [94]:
acc10 = acc(model_10,valid_feature,valid_label);print acc10
acc50 = acc(model_50,valid_feature,valid_label);print acc50
acc100 = acc(model_100,valid_feature,valid_label);print acc100
acc200 = acc(model_200,valid_feature,valid_label);print acc200
acc500 = acc(model_500,valid_feature,valid_label);print acc500

0.666307626023
0.681602757432
0.689142610944
0.68935803533
0.689250323137
